In [ ]:
'''
PPCA 3.0 MORPHOMETRY ON OSM BUILDINGS

Author : Perez, Joan 

This script performs several calculations and transformations on a layer of OSM buildings. It begins by ensuring the columns 'height' and 'building
are numeric, converting any non-numeric entries to NaN. The script then fills missing 'height' values by multiplying floors by 3, assuming an 
average floor height of 3 meters. Conversely, it fills missing building values by dividing 'height' by 3 and rounding the result. It calculates and
prints the number and percentage of rows with NaN in both 'height' and 'building. Several new columns are computed: 'FL' for the number of floors,
'A' for the surface area, 'P' for the perimeter, 'E' for elongation, 'C' for convexity, 'FA' for floor area, 'ECA' for a product involving 
elongation, convexity, and area, 'EA' for another elongation-area product, and 'SW' for shared walls ratio. Finally, the script renames 
'building:floors' to 'FL'.

Requirements:
- A specific working environment (see README on the github page of the project https://github.com/perezjoan/PPCA-codes?tab=readme-ov-file)
- Output file from PPCA 1.0 (Geopackage)

Guide to run the script:
- Fill 0.2 box

Output :
- A geopackage file with a single layer
    * 'osm_all_buildings_ind' (Polygon), osm buildings with height/floor values completed and with morphometric indicators

Acknowledgement: This resource was produced within the emc2 project, which is funded by ANR (France), FFG (Austria), MUR (Italy) and 
Vinnova (Sweden) under the Driving Urban Transition Partnership, which has been co-funded by the European Commission.

License: Attribution-ShareAlike 4.0 International - CC-BY-SA-4.0 license
'''

In [1]:
# 0.1 : libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import momepy
import libpysal
import warnings

In [2]:
# 0.2 : Box to fil with informations

# Name of the case study
Name = 'Nice'

# Define projected CRS
projected_crs = 'EPSG:2154'

In [4]:
# 1. FILL & CALCULATE MISSING VALUES FOR HEIGHT & LEVEL
# Import OSM buildings
gpkg = f'PPCA1_{Name}.gpkg'
building = gpd.read_file(gpkg, layer = 'osm_all_buildings')

# Ensure height & level columns are numeric
building['height'] = pd.to_numeric(building['height'], errors='coerce')
building['building:levels'] = pd.to_numeric(building['building:levels'], errors='coerce')

# checks if height is NaN and if floor is not NaN.
# If both conditions are met : multiplies the value of floor by 3 and assigns it to height
building['height'] = building.apply(lambda row: row['building:levels'] * 3 if pd.isna(row['height'])
                                     and not pd.isna(row['building:levels']) else row['height'], axis=1)
# checks if the height column is NaN and if the floor is not NaN.
# If both conditions are met : multiplies the value of floor by 3 and assigns it to height
building['building:levels'] = building.apply(lambda row: round(row['height'] / 3) if pd.isna(row['building:levels'])
                                             and not pd.isna(row['height']) else row['building:levels'], axis=1)

# Calculate the number and percentage of rows with NaN for both 'height' and 'building:levels'
num_na = building[['height', 'building:levels']].isna().all(axis=1).sum()
percent_na = (num_na / len(building)) * 100

# Print the number and percentage of rows with NA for both columns
print(f'{Name} : Number of remaining values with NA for height/floors: {num_na} ({percent_na:.2f}%)')

Nice : Number of remaining values with NA for height/floors: 99057 (66.17%)


In [6]:
# 2. CALCULATE MORPHOMETRIC INDICATORS FOR EACH BUILDING
# Go to a projected CRS
building = building.to_crs(projected_crs)

# Number of floors
building.rename(columns={'building:levels': 'FL'}, inplace=True)

# Surface
building['A'] = building.geometry.area

# Calculating perimeter
building['P'] = building.geometry.length

# Calculating elongation
building['E'] = momepy.Elongation(building).series

# Convexity
building['C'] = momepy.Convexity(building).series

# Floor area
building['FA'] = building['FL'] * building['A']

# Product [1-E].C.S
building['ECA'] = (1 - building['E']) * building['A'] * building['C']

# [1-E].S
building['EA'] = (1 - building['E']) * building['A']

# Shared walls
warnings.filterwarnings("ignore", category=FutureWarning, module="momepy")
building["SW"] = momepy.SharedWallsRatio(building).series

In [7]:
# Save & plot
gpkg = f'PPCA3_{Name}.gpkg'
building.to_file(gpkg, layer='osm_all_buildings_ind', driver="GPKG")